# Getting Started

This notebook shows how to work with `hephaistos`.

In [1]:
import hephaistos
import numpy as np

We start by enumerating all available devices:

In [2]:
assert hephaistos.isVulkanAvailable(), "Vulkan is not available on your system!"

for i, device in enumerate(hephaistos.enumerateDevices()):
    print(f'{i}: {device.name}{" (discrete)" if device.isDiscrete else ""}')

0: NVIDIA GeForce RTX 3060 Ti (discrete)


## Copying stuff around

Here we show how copying data from and to the gpu works, and how commands can be sequenced.
Hephaistos has two major data types: `Buffer`s living on the CPU and `Tensor`s living on the GPU.

In [3]:
buffer1 = hephaistos.IntBuffer(10)
np.copyto(buffer1.numpy(), np.arange(10))

In [4]:
tensor = hephaistos.IntTensor(10)
buffer2 = hephaistos.IntBuffer(10)

In [5]:
timeline = hephaistos.Timeline()
copy = hephaistos.beginSequence(timeline) \
    .And(hephaistos.updateTensor(buffer1, tensor)) \
    .Then(hephaistos.retrieveTensor(tensor, buffer2)) \
    .Submit()

In [6]:
print("Uploading Data...")
timeline.wait(1)
print("Fetching Data...")
timeline.wait(2)
print("Done")

Uploading Data...
Fetching Data...
Done


In [7]:
buffer2.numpy()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

## Program / Compute Shader

In [8]:
source = """
#version 460

layout(local_size_x = 1) in;

layout(binding = 0) readonly buffer tensorA { int in_a[]; };
layout(binding = 1) readonly buffer tensorB { int in_b[]; };
layout(binding = 2) writeonly buffer tensorOut { int out_c[]; };

void main() {
    uint idx = gl_GlobalInvocationID.x;
    out_c[idx] = in_a[idx] + in_b[idx];
}
"""

code = hephaistos.compileSource(source)
program = hephaistos.Program(code)

In [9]:
#reuse a single buffer for staging
buffer = hephaistos.IntBuffer(10)
buffer.numpy()[:] = [15, 36, 51, 12, 99, 102, 12, 33, 54, 22]

tensorA = hephaistos.IntTensor(10)
tensorB = hephaistos.IntTensor(10)
tensourOut = hephaistos.IntTensor(10)

hephaistos.bindParams(program, [tensorA, tensorB, tensourOut])

# You could also reuse timeline but than you have to keep track of its value
timeline = hephaistos.Timeline()
submission = hephaistos.beginSequence(timeline) \
    .And(hephaistos.updateTensor(buffer, tensorA)) \
    .WaitFor(2).And(hephaistos.updateTensor(buffer, tensorB)) \
    .Then(program.dispatch(10)) \
    .Then(hephaistos.retrieveTensor(tensourOut, buffer)) \
    .Submit()

timeline.wait(1)
buffer.numpy()[:] = [13, 12, 28, 23, 94, 56, 72, 28, 44, 13]
timeline.value = 2
submission.wait()

buffer.numpy()

array([ 28,  48,  79,  35, 193, 158,  84,  61,  98,  35], dtype=int32)